In [3]:
#load the libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import string


In [4]:
# Load the dataset
df = pd.read_csv('Language Detection.csv')

def clean(text):
    for pun in string.punctuation:
        text = text.replace(pun, '')
    text = text.lower()
    return text

df['Text']=df['Text'].apply(clean)

# Split the dataset into input features and labels
X = df.iloc[:, 0].values
y = df.iloc[:, 1].values

In [5]:
df

,Text,Language
0,nature in the broadest sense is the natural p...,English
1,nature can refer to the phenomena of the physi...,English
2,the study of nature is a large if not the only...,English
3,although humans are part of nature human activ...,English
4,1 the word nature is borrowed from the old fre...,English
...,...,...
10332,ನಿಮ್ಮ ತಪ್ಪು ಏನು ಬಂದಿದೆಯೆಂದರೆ ಆ ದಿನದಿಂದ ನಿಮಗೆ ಒ...,Kannada
10333,ನಾರ್ಸಿಸಾ ತಾನು ಮೊದಲಿಗೆ ಹೆಣಗಾಡುತ್ತಿದ್ದ ಮಾರ್ಗಗಳನ್...,Kannada
10334,ಹೇಗೆ ನಾರ್ಸಿಸಿಸಮ್ ಈಗ ಮರಿಯನ್ ಅವರಿಗೆ ಸಂಭವಿಸಿದ ಎಲ...,Kannada
10335,ಅವಳು ಈಗ ಹೆಚ್ಚು ಚಿನ್ನದ ಬ್ರೆಡ್ ಬಯಸುವುದಿಲ್ಲ ಎಂದು ...,Kannada


In [6]:
# Encode non-numerical labels using LabelEncoder
if not np.issubdtype(y.dtype, np.number):
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the input text
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1

In [7]:
# Convert text sequences to padded sequences
max_length = 200
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

In [8]:
# Convert labels to categorical
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [13]:
#Design the ANN
embedding_dim = 200
inputs = Input(shape=(max_length,))
x = Embedding(vocab_size, embedding_dim)(inputs)
x = LSTM(256, return_sequences=True)(x)
x = LSTM(256)(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

In [14]:
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
# Train the model
with tf.device('/GPU:0'):  # Specify the GPU device
    model.fit(X_train_padded, y_train, validation_data=(X_test_padded, y_test), epochs=1000, batch_size=64)


Epoch 1/1000
 39/130 [========>.....................] - ETA: 4:16 - loss: 2.7597 - accuracy: 0.1262

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")